# How to create custom Document links

Graph vector stores allow documents to be connected in different ways, allowing different types of relationships between documents to be used in retrieval.

Several common types of relationships can be defined automatically, but in some cases you may want to create custom links. In this example we'll see how to create links directly.

In this guide we will cover:

1. How document links work;
2. How to create custom links;
2. How to extend a document custom links;
3. How to retrieve documents from a graph vector store.

## How document links work

Documents in a graph vector store are connected via "links". Links form a bipartite graph between documents and tags: documents are connected to tags, and tags are connected to other documents. When documents are retrieved from a graph vector store, a pair of documents are connected with a depth of one if both documents are connected to the same tag.

Links have a "kind", used to namespace different tag identifiers. For example a link to a keyword might use kind `kw`, while a link to a URL might use kind `url`. This allows the same tag value to be used in different contexts without causing name collisions.

Links are directed - the directionality of links controls how the graph is traversed at retrieval time. For example, given documents A and B, connected by links to tag T:

| A to T | B to T | Result |
|--------|--------|--------|
| outgoing | incoming | Retrieval traverses from A to B |
| incoming | incoming | No traversal from A to B |
| outgoing | outgoing | No traversal from A to B |
| incoming | outgoing | No traversal from A to B |
| bidir | incoming | Retrieval traverses from A to B |
| bidir | outgoing | No traversal from A to B |
| outgoing | bidir | Retrieval traverses from A to B |
| incoming | bidir | No traversal from A to B |

Directed links make it possible to describe relationships such as term references / definitions: term definitions are generally relevant to any documents that use the term, but the full set of documents using a term generally aren't relevant to the term's definition.

## How to create custom links

You can create links using the [Link](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html) class's constructors [.bidir](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html#langchain_core.graph_vectorstores.links.Link.bidir), [.incoming](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html#langchain_core.graph_vectorstores.links.Link.incoming), and [.outgoing](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html#langchain_core.graph_vectorstores.links.Link.outgoing).

In [ ]:
from langchain_core.graph_vectorstores.links import Link

link = Link.bidir(kind="location", tag="Paris")

## Extending documents with custom links

Now that we know how to create links, let's associate them with some documents. These edges will strengthen the connection between documents that share a keyword when using a graph vector store to retrieve documents.

First, we'll load some text and chunk it into smaller pieces.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.graph_vectorstores.extractors import KeybertLinkExtractor

loader = TextLoader("state_of_the_union.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

keyword_extractor = KeybertLinkExtractor()
keyword_extractor.extract_one(texts[0])

In [ ]:
from langchain_core.graph_vectorstores.links import add_links

for text in texts:
    add_links(text, Link.bidir(kind="genre", tag="oratory"))

texts[0].metadata

As you can see, each document's metadata now includes a bidirectional link to the genre "oratory".

## Retrieving documents from a graph vector store

In order to use links during document retrieval we'll first add our documents to a graph vector store, then query the store as a retriever.

We'll begin by adding the documents to a graph vector store. We'll use a store backed by Cassandra [CassandraGraphVectorStore](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_community.graph_vectorstores.cassandra.CassandraGraphVectorStore.html) graph vectorstore:

In [ ]:
from langchain_community.graph_vectorstores import CassandraGraphVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
graph_vectorstore = CassandraGraphVectorStore.from_documents(texts, embeddings)

We can then instantiate a retriever:

In [ ]:
retriever = graph_vectorstore.as_retriever()

This creates a retriever (specifically a [GraphVectorStoreRetriever](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.base.GraphVectorStoreRetriever.html)), which we can use in the usual way:

In [3]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")